# EDA 
series_idの確認

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
sys.path.append("/kaggle/src/dss_utils")

from dss_metrics import score

# Config

In [ ]:
output_dir = os.path.join("/kaggle", "working", "_oof")
exp_name = "exp006_addlayer"

In [ ]:
for i in range(5):
    print("fold", i)
    oof_df = pd.concat(
        [pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))]
        ,axis=0
    )


In [ ]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")

In [ ]:
series_df = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")

In [ ]:
print(set(series_df["series_id"].unique()) - set(train_event_df["series_id"].unique()))
print(set(train_event_df["series_id"].unique()) - set(series_df["series_id"].unique()))

print(set(series_df["series_id"].unique()) - set(oof_df["series_id"].unique()))
print(set(oof_df["series_id"].unique()) - set(series_df["series_id"].unique()))

print(set(train_event_df["series_id"].unique()) - set(oof_df["series_id"].unique()))
print(set(oof_df["series_id"].unique()) - set(train_event_df["series_id"].unique()))


In [ ]:
print(len(set(series_df["series_id"].unique()) - set(oof_df["series_id"].unique())))
print(len(set(train_event_df["series_id"].unique())- set(oof_df["series_id"].unique())))

In [ ]:
preprocessed_df = pd.read_parquet("/kaggle/input/preprocessed_train_series_le_fold.parquet")

In [ ]:
print(set(series_df["series_id"].unique()) - set(preprocessed_df["series_id"].unique()))
print(set(preprocessed_df["series_id"].unique()) - set(series_df["series_id"].unique()))

In [ ]:
key_df = preprocessed_df[["series_date_key", "series_date_key_str"]].drop_duplicates()
key_df = key_df.reset_index(drop=True)
key_df["series_id"], key_df["date"] = (
    key_df["series_date_key_str"].str.split("_", 1).str
)
key_df = key_df.drop(columns=["series_date_key_str"], axis=1)

In [ ]:
print(set(key_df["series_id"].unique()) - set(series_df["series_id"].unique()))
print(set(series_df["series_id"].unique()) - set(key_df["series_id"].unique()))

In [ ]:
not_include_series = list(set(series_df["series_id"].unique()) - set(oof_df["series_id"].unique()))
print(len(not_include_series))

In [ ]:
display_col = ["series_id", "series_date_key", "fold", "step"]
for idx, series_id in enumerate(not_include_series):
    if idx > 10:
        break
    print(series_id)
    df = preprocessed_df[preprocessed_df["series_id"] == series_id]
    display(df[display_col])
    plt.figure(figsize=(10, 5))
    plt.subplot(3, 1, 1)
    plt.plot(df["anglez"])
    plt.subplot(3, 1, 2)
    plt.plot(df["enmo"])
    plt.subplot(3, 1, 3)
    plt.plot(df["event"])
    plt.show()
    